In [154]:
import os
from os import path
from datetime import datetime, timedelta, date
import pytz

# How to get igraph to work with conda:
# 1) Create separate environment
# 2) Install all neccesary dependencies through conda, except cairo, pycairo, and igraph. 
#    From a freshly created conda environment, you would only need pytz, and Pillow
# 3) Install cairo and ffmpeg with brew (brew install cairo, brew install ffmpeg)
# 4) Install pycairo and igraph with pip (pip install pycairo, pip install igraph)
from igraph import *

from PIL import Image, ImageDraw, ImageFont

import seaborn as sns
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as clr
%matplotlib inline

In [155]:
# Various datasets to visualize

option = 10
time0 = time1 = ''
use_new_id_schema = False

if option == 1:
    base_folder = "./simulations/retreat2020"
    sim_id = 29
    sim_tz = "US/Eastern"
    time_step_min = 30
if option == 2:
    base_folder = "./simulations/cmutest"
    sim_id = 32
    sim_tz = "US/Mountain"
    time_step_min = 30
if option == 3:
    base_folder = "./simulations/cmu"
    sim_id = 34
    sim_tz = "US/Mountain"
    time_step_min = 60
if option == 4:
    base_folder = "./simulations/fgcu"
    sim_id = 33
    sim_tz = "US/Eastern"
    time_step_min = 60
if option == 5:
    base_folder = "./simulations/csw1"
    sim_id = 36
    sim_tz = "US/Eastern"
    time_step_min = 60
if option == 6:
    base_folder = "./simulations/csw2"
    sim_id = 37
    sim_tz = "US/Eastern"
    time_step_min = 60
if option == 7:
    base_folder = "./simulations/byu21"
    sim_id = 39
    sim_tz = "US/Mountain"
    time_step_min = 60
if option == 8:
    title = "UCAS21 OO SIMULATION"
    base_folder = "./simulations/ucas21"
    sim_id = 40
    sim_tz = "US/Mountain"
    time_step_min = 60
    
if option == 10:
    title = "WALTER JOHNSON HIGH SCHOOL"
    base_folder = "./simulations/wjhs22"
    sim_id = 76
    sim_tz = "US/Eastern"
    time0 = 'Oct 14 2022 8:15AM'
    time1 = 'Oct 14 2022 8:20AM'
    time_step_min = 1
    use_new_id_schema = True    

In [166]:
# Configuration

# Coded status:
# https://matplotlib.org/3.1.0/gallery/color/named_colors.html
status_color = {0: clr.to_hex("skyblue"),  # no infection
                1: clr.to_hex("crimson"),  # infected, index case
                2: clr.to_hex("crimson"),  # infected, from someone else
                3: clr.to_hex("gray"),     # silver 
                4: clr.to_hex("skyblue"),  # recovered 
                5: clr.to_hex("skyblue")   # orchid 
               } 

# https://github.com/google/fonts/tree/master/apache
label_font = ImageFont.truetype("Roboto-Regular.ttf", size=24)

# Visual style of the contact & infection networks
# https://igraph.org/python/versions/latest/tutorial.html#vertex-attributes-controlling-graph-plots

cstyle = {}
cstyle["bbox"] = (1200, 800)
cstyle["margin"] = 15
cstyle["vertex_size"] = 7
cstyle["vertex_frame_width"] = 0
cstyle["vertex_label_size"] = 5
cstyle["edge_curved"] = False
cstyle["edge_width"] = 0

istyle = {}
istyle["bbox"] = (1200, 800)
istyle["margin"] = 15
istyle["vertex_size"] = 7
istyle["vertex_frame_width"] = 0
istyle["vertex_label_size"] = 5
istyle["edge_color"] = "crimson"
istyle["edge_arrow_size"] = 0.6
istyle["edge_arrow_width"] = 0.6
istyle["edge_curved"] = True

data_folder = path.join(base_folder, "data")
output_folder = path.join(base_folder, "output", "contacts")
if not path.exists(output_folder):
    os.makedirs(output_folder)

# Print warning messages to the console when parsing data
print_data_warnings = False
    
# Default contact time for transmissions that are missing an associated contact event
def_contact_time = 10
    
frame_format = "png"
movie_fps = 2

# Time delta for plots in seconds
time_delta_sec = 60 * time_step_min

# Parameters of the layout algorithm, the anim_steps is how many times the fruchterman-reingold (fr)
# algorithm is run per time delta, the higher the smoother the animation will be.
# fr_niter controls the number of iterations to perform by the fr algorithm.
# The product of these two numbers should be around 200 ~ 500
anim_steps_per_time_delta = 30
fr_niter = 10

# https://howchoo.com/g/ywi5m2vkodk/working-with-datetime-objects-and-timezones-in-python
# https://itnext.io/working-with-timezone-and-python-using-pytz-library-4931e61e5152
timezone = pytz.timezone(sim_tz)

if time0 and time1:
    obs_date0 = timezone.localize(datetime.strptime(time0, '%b %d %Y %I:%M%p'))
    obs_date1 = timezone.localize(datetime.strptime(time1, '%b %d %Y %I:%M%p'))
else:
    obs_date0 = None
    obs_date1 = None

In [167]:
# Some utility functions

def get_contact_list(events, infections):
    contacts = events[events["type"] == "contact"]

    node0 = contacts.user_id.values
    node1 = contacts.peer_id.values
    length = contacts.contact_length.values

    clist = {}
    for id0, id1, l01 in zip(node0, node1, length):
        n0 = user_index[id0]
        if use_new_id_schema:
            if id1 in user_index:
                n1 = user_index[id1]
            elif print_data_warnings:
                print("Cannot find peer", id1)
        else:
            if id1 in p2pToId:
                n1 = user_index[p2pToId[id1]]
            elif print_data_warnings:
                print("Cannot find peer", id1)
    
        if n0 < n1:
            p01 = (n0, n1)
        else:
            p01 = (n1, n0)
        if p01 in clist:
            c = clist[p01]
        else: 
            c = 0

        clist[p01] = c + round(l01 / (60 * 1000))
                        
    # Adding contacts from transmissions if they are not registered as contacts already
    for (n0, n1) in infections:
        if n0 < n1:
            p01 = (n0, n1)
        else:
            p01 = (n1, n0)
        if not p01 in clist:
            clist[p01] = def_contact_time
            if print_data_warnings: print("Cannot find contact between", n0, "and", n1)            

    return clist

def get_infection_list(events):
    infections = events[(events["type"] == "infection")]

    ilist = []
    infected = infections.user_id.values
    peers = infections.inf.values
    for id1, peer0 in zip(infected, peers):
        n1 = user_index[id1]

        if "PEER" in peer0:
            if use_new_id_schema:
                # New schema
                id0 = int(peer0[peer0.index("[") + 1:peer0.index(":")])
                if id0 in user_index:
                    n0 = user_index[id0]
                    ilist += [(n0, n1)]
                elif print_data_warnings:
                    print("Cannot find peer", id0)

            else:
                # Old schema (sims before 2022): p2p id is in the infection column
                p2p0 = peer0[peer0.index("[") + 1:peer0.index(":")]
                if p2p0 in p2pToId:
                    id0 = p2pToId[p2p0]
                    if id0 in user_index:
                        n0 = user_index[id0]
                        ilist += [(n0, n1)]
                    elif print_data_warnings:
                        print("Cannot find peer", id0)
            
    return ilist 

def get_node_status(events, status0 = None):    
    if status0 == None:
         status = [0] * len(users)
    else:            
        status = status0

    inf = events[events["type"] == "infection"]
    infMap = pd.Series(inf.inf.values, index=inf.user_id).to_dict()
    for kid in infMap:
        src = infMap[kid]
        idx = user_index[kid]
        if "CASE0" in src:
            status[idx] = 1
        if "PEER" in src:
            status[idx] = 2

    out = events[events["type"] == "outcome"]
    outMap = pd.Series(out.out.values, index=out.user_id).to_dict()
    for kid in outMap:
        out = outMap[kid]
        idx = user_index[kid]
        if out == "DEAD":
            status[idx] = 3
        if out == "RECOVERED":
            status[idx] = 4
        if out == "VACCINATED":
            status[idx] = 5
    
    # TODO: read modifiers (mask, PPE)
    #mod = events[events["modifier"] != ""]
    #modMap = pd.Series(mod.modifier.values, index=mod.user_id).to_dict()    
    # modifier
    # int
    # "Wearing PPE"
    # "Wearing Mask"
    
    return status

def get_contact_network(contacts, status):
    nvert = len(user_index)

    edges = []
    weights = []
    edgesw = []
    
    if 0 < len(contacts):
        for p in contacts:
            n0 = p[0]
            n1 = p[1]
            w = contacts[p]    
            if 0 < w:
                edges += [(n0, n1)]
                weights += [w]
                edgesw += [(n0, n1, w)]

    # https://stackoverflow.com/a/50430444
    g = Graph(directed=False)
    g.add_vertices(nvert)
    g.add_edges(edges)
    g.es['weight'] = weights

    if status:
        g.vs["status"] = status
        g.vs["color"] = [status_color[out] for out in g.vs["status"]]
    
    return g

def get_infection_network(infections, status):
    nvert = len(user_index)
    
    g = Graph(directed=True)
    g.add_vertices(nvert)
    g.add_edges(infections)

    if status:
        g.vs["status"] = status
        g.vs["color"] = [status_color[out] for out in g.vs["status"]]
    
    return g

def gen_layout(g):
    # https://igraph.org/c/doc/igraph-Layout.html#igraph_layout_graphopt
    return g.layout_fruchterman_reingold(weights=g.es["weight"])

def plot_network(g, style, layout, title, fn):
    img_fn = os.path.join(output_folder, fn)
    
    style["layout"] = layout
    p = plot(g, img_fn, **style)
    
    if ".png" in fn and title:
        image = Image.open(img_fn)
        draw = ImageDraw.Draw(image)
        draw.text((10, 760), title, fill='rgb(0, 0, 0)', font=label_font)
        image.save(img_fn)        

def print_network_properties(g):
    print("Number of vertices in the graph:", g.vcount())
    print("Number of edges in the graph", g.ecount())
    print("Is the graph directed:", g.is_directed())
    print("Maximum degree in the graph:", g.maxdegree())
#     print("Adjacency matrix:\n", g.get_adjacency())

# https://stackoverflow.com/a/48938464
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

In [168]:
# Load participants and histories

all_users = pd.read_csv(path.join(data_folder, "participants.csv")) 
all_events = pd.read_csv(path.join(data_folder, "histories.csv"))

users = all_users[all_users["sim_id"] == sim_id]

events = all_events[all_events["sim_id"] == sim_id]
events.fillna({'contact_length':0, 'peer_id':-1}, inplace=True)
events["event_start"] = events["time"] - events["contact_length"]/1000
events["event_start"] = events["event_start"].astype(int, errors = 'ignore')
if use_new_id_schema:
    events["peer_id"] = events["peer_id"].astype(int, errors = 'ignore')

p2pToSim = pd.Series(users.sim_id.values, index=users.p2p_id).to_dict()
p2pToId = pd.Series(users.id.values, index=users.p2p_id).to_dict()
idTop2p = pd.Series(users.p2p_id.values, index=users.id).to_dict()
        
user_index = {}
idx = 0
for kid in idTop2p:
    user_index[kid] = idx
    idx += 1

# These should return the same value
print(len(users))
print(len(idTop2p))    
print(len(p2pToId))
print(len(user_index))

/var/folders/4g/z3rj5j_j781_8ckvkcy7p4840000gp/T/ipykernel_91869/4203906309.py:4: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  all_events = pd.read_csv(path.join(data_folder, "histories.csv"))


446
446
446
446


In [169]:
status = get_node_status(events)
infections = get_infection_list(events)
contacts = get_contact_list(events, infections)
g = get_contact_network(contacts, status)
print_network_properties(g)

Cannot find contact between 106 and 27
Cannot find contact between 44 and 0
Number of vertices in the graph: 446
Number of edges in the graph 14605
Is the graph directed: False
Maximum degree in the graph: 190


In [170]:
# Round min and max times to the hour
min_time = min(events['time'])
max_time = max(events['time'])
first_date = hour_rounder(datetime.fromtimestamp(min_time, tz=timezone))
last_date = hour_rounder(datetime.fromtimestamp(max_time, tz=timezone))
min_time = datetime.timestamp(first_date)
max_time = datetime.timestamp(last_date)

print("First event:", first_date)
print("Last event :", last_date)

if time0 and time1:
    print("Start time:", datetime.strptime(time0, '%b %d %Y %I:%M%p'))
    print("End time:", datetime.strptime(time1, '%b %d %Y %I:%M%p'))

print(first_date.tzinfo)

First event: 2022-10-05 16:00:00-04:00
Last event : 2022-10-15 13:00:00-04:00
Start time: 2022-10-14 08:15:00
End time: 2022-10-14 08:20:00
US/Eastern


In [171]:
# Contacts over time

print("CREATING FRAMES...") 

# How to properly animate an igraph network over time (so nodes change position smoothly from frame to frame):
# http://estebanmoro.org/post/2015-12-21-temporal-networks-with-r-and-igraph-updated/
# https://github.com/emoro/temporal_networks

frame = 0
img_array = []
layout0 = None
gi0 = None
tstatus = None

if obs_date0 and obs_date1:
    tmin = datetime.timestamp(obs_date0)
    tmax = datetime.timestamp(obs_date1)
else:
    tmin = min_time
    tmax = max_time

t = tmin
print("FRAME", end =" ") 
while t <= tmax:
    t0 = t
    t += time_delta_sec
    td = datetime.fromtimestamp(t, tz=timezone)    

    # We want to include contatact and infection events that either started or ended between t0 and t
    condition = ((t0 < events['event_start']) & (events['event_start'] <= t)) | ((t0 < events['time']) & (events['time'] <= t))
    
    tevents = events[condition]
    tstatus = get_node_status(tevents, tstatus)
    tinfections = get_infection_list(tevents)
    tcontacts = get_contact_list(tevents, tinfections)

    gc = get_contact_network(tcontacts, tstatus)
    gi = get_infection_network(tinfections, tstatus)

    for i in range(0, anim_steps_per_time_delta):
        print(frame, end =" ")

        # https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#layout_fruchterman_reingold
        layout = gc.layout_fruchterman_reingold(niter=fr_niter, start_temp=0.05, grid='nogrid', weights=gc.es["weight"], seed=layout0)
        layout0 = layout.copy()
    
        img_title = td.strftime('%B %d, %I:%M %p')
        img_fn =  "frame-" + str(frame) + "." + frame_format
        plot_network(gi, istyle, layout, img_title, img_fn)
    
        frame += 1
        
print("\nDONE")

CREATING FRAMES...
FRAME 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 Cannot find contact between 254 and 238
Cannot find contact between 44 and 0
Cannot find contact between 6 and 312
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 
DONE


In [172]:
print("CREATING THE MOVIE FILE...")

movie_fn = path.join(output_folder, "movie.mp4")
if path.exists(movie_fn):
    os.remove(movie_fn)

cmd_str = "ffmpeg -i " + output_folder + "/frame-%d.png -c:v libx264 -pix_fmt yuv420p " + output_folder + "/movie.mp4"
os.system(cmd_str)

print("DONE")

CREATING THE MOVIE FILE...


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.102)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox


DONE


frame=  180 fps=0.0 q=-1.0 Lsize=     920kB time=00:00:07.08 bitrate=1064.7kbits/s speed=9.25x    
video:918kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.261750%
[libx264 @ 0x7f8f77f07100] frame I:1     Avg QP:25.13  size: 19966
[libx264 @ 0x7f8f77f07100] frame P:115   Avg QP:24.02  size:  5689
[libx264 @ 0x7f8f77f07100] frame B:64    Avg QP:25.49  size:  4139
[libx264 @ 0x7f8f77f07100] consecutive B-frames: 45.0% 20.0%  8.3% 26.7%
[libx264 @ 0x7f8f77f07100] mb I  I16..4:  2.3% 77.9% 19.7%
[libx264 @ 0x7f8f77f07100] mb P  I16..4:  0.7%  6.7%  1.5%  P16..4: 17.5%  4.3%  1.3%  0.0%  0.0%    skip:68.1%
[libx264 @ 0x7f8f77f07100] mb B  I16..4:  0.4%  0.9%  1.1%  B16..8: 27.4%  4.6%  0.7%  direct: 0.2%  skip:64.8%  L0:53.8% L1:45.1% BI: 1.1%
[libx264 @ 0x7f8f77f07100] 8x8 transform intra:71.2% inter:26.8%
[libx264 @ 0x7f8f77f07100] coded y,uvDC,uvAC intra: 10.9% 23.4% 22.1% inter: 1.4% 3.7% 2.8%
[libx264 @ 0x7f8f77f07100] i16 v,h,dc,p: 69% 25%  6%  0%
[li